In [1]:
import json

# Enter your keys/secrets as strings in the following fields
credentials = {}
 #tokennya semua diubahcredentials['CONSUMER_KEY'] = '0Q4CL7UPfuQ2HqVLqqzmBgc9'
credentials['CONSUMER_SECRET'] = 'zTsDGhqOakBTdlIIr25j1LLcHugDVWUsWQyjMJAj3PSh3VpBK'
credentials['ACCESS_TOKEN'] = '825116012-65Y1lVc6AcDCZsbfkQDrjdvMTkYvQPdnz5pnyke'
credentials['ACCESS_SECRET'] = 'uBVx8JUkAKp9VtIcnIIJUwka5DlhYxbBkXqGxb4OFZIU'

# Save the credentials object to file
with open("twitter_credentials.json", "w") as file:
    json.dump(credentials, file)

In [16]:
# Import the Twython class
from twython import Twython
import json

# Load credentials from json file
with open("twitter_credentials.json", "r") as file:
    creds = json.load(file)

# Instantiate an object
python_tweets = Twython(creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'])

# Create our query
query = {'q': 'corona',
        }

In [17]:
import pandas as pd

# Search tweets
dict_ = {'user': [], 'date': [], 'text': [], 'favorite_count': []}
for status in python_tweets.search(**query)['statuses']:
    dict_['user'].append(status['user']['screen_name'])
    dict_['date'].append(status['created_at'])
    dict_['text'].append(status['text'])
    dict_['favorite_count'].append(status['favorite_count'])

# Structure data in a pandas DataFrame for easier manipulation
df = pd.DataFrame(dict_)
df.sort_values(by='favorite_count', inplace=True, ascending=False)
df.head(5)

,user,date,text,favorite_count
0,Dik_siyonner,Mon Mar 16 07:04:52 +0000 2020,RT @Ozguruz_org: Can Dündar'ın #ÖzgürYorum'u h...,0
1,Mhmdshht27,Mon Mar 16 07:04:52 +0000 2020,RT @tamamdanie: corona virüs hala türkiyeye ge...,0
2,mengsyoun,Mon Mar 16 07:04:52 +0000 2020,RT @azwarpahreza: @ErsaTriWahyuni Untuk kesela...,0
3,hamsaankit,Mon Mar 16 07:04:52 +0000 2020,RT @hamsaankit: @globaltimesnews \nWhy did Chi...,0
4,RanaBardhan3,Mon Mar 16 07:04:52 +0000 2020,"RT @OpIndia_in: ‘हमें कोरोना नहीं होगा, मोदी क...",0


In [13]:
df

,user,date,text,favorite_count
4,CNNIndonesia,Sat Mar 14 10:26:09 +0000 2020,Pemerintah Akui Petugas Medis Meninggal Terinf...,32359
2,CNNIndonesia,Sat Mar 14 09:14:43 +0000 2020,"Update Corona 14 Maret: 96 Positif, 5 Meningga...",27970
1,CNNIndonesia,Sun Mar 15 06:44:17 +0000 2020,"#DataTerbaruCorona\n21 Kasus Baru, Pasien Posi...",23291
3,CNNIndonesia,Sun Mar 15 10:56:06 +0000 2020,Bayi Baru Lahir di London Terinfeksi Virus Cor...,20610
0,CNNIndonesia,Sat Mar 14 13:59:34 +0000 2020,Istana: Menhub Budi Karya Positif Corona https...,16624
5,detikcom,Sat Mar 14 14:11:54 +0000 2020,"Menhub Budi karya Sumadi positif corona, tugas...",7121
7,CNNIndonesia,Sun Mar 15 02:27:05 +0000 2020,Corona dan Menanti Maaf Jokowi untuk Rakyat +6...,5026
9,CNNIndonesia,Sun Mar 15 06:49:35 +0000 2020,"21 Kasus Baru, Pasien Positif Corona Indonesia...",4882
8,Haidar_Bagir,Sun Mar 15 06:07:41 +0000 2020,Semua hrs nonton ini. Penjelasan sangat bagus ...,2765
6,kumparan,Sat Mar 14 13:52:02 +0000 2020,Menteri @kemenhub151 Budi Karya Sumadi dinyata...,1981


In [21]:
from twython import TwythonStreamer
import csv

# Filter out unwanted data
def process_tweet(tweet):
    d = {}
    d['hashtags'] = [hashtag['text'] for hashtag in tweet['entities']['hashtags']]
    d['text'] = tweet['text']
    d['user'] = tweet['user']['screen_name']
    d['user_loc'] = tweet['user']['location']
    return d
    
    
# Create a class that inherits TwythonStreamer
class MyStreamer(TwythonStreamer):     

    # Received data
    def on_success(self, data):

        # Only collect tweets in English
        if data['lang'] == 'en':
            tweet_data = process_tweet(data)
            self.save_to_csv(tweet_data)

    # Problem with the API
    def on_error(self, status_code, data):
        print(status_code, data)
        self.disconnect()
        
    # Save each tweet to csv file
    def save_to_csv(self, tweet):
        with open(r'saved_tweets.csv', 'a', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow(list(tweet.values()))

In [22]:
# Instantiate from our streaming class
stream = MyStreamer(creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'], 
                    creds['ACCESS_TOKEN'], creds['ACCESS_SECRET'])
# Start the stream
stream.statuses.filter(track='corona')

KeyError: 'lang'